# Convolutional Neural Networks
---
The reason why CNNs work so well is because of 2 properties observed in data:
* __Stationarity of Statistics:__ This means given small local patches, there is a high probability of finding recurrent patterns in the locality. A simpler definition for this is that some motifs tend to reoccur within the (image) data. This allows for parameter sharing in case of CNNs, which inversely affects the number of parameters in the model, and hence allows relatively lesser training times as compared to, say, fully connected layers.

* __Locality of Pixel Dependencies:__ This principal states that pixels that are close to each other tend to be more correlated and dependent on each other as compared to those far away. In simpler words, pixels that are closer to each other tend to be of similar color. This also means that related data tends to be concentrated into small patches. Locality affects that sparsity of the connections.

* __Compositionality:__ Talking about in terms of image data, images are composed of smaller, simpler patterns. In fact, all data is composed of simpler data. Thus, instead of looking for a certain object within the image, the network can focus on discovering these patterns within the image. 

In this notebook, we will see how CNNs perform as compared to FC nerworks.

In [1]:
from res.plot_lib import *
set_default() # setting the default plot style

In [2]:
# importing dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

In [3]:
# function to calculate the number of parameters in the model
def count_parameters(model):
    params = 0
    for p in list(model.parameters()):
        params += p.nelement()
    return params

In [4]:
# selecting the default device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
# creating the dataloaders to feed data to the models
train_loader = DataLoader(
    datasets.MNIST("E-Learning/NYU-DL/data/MNIST", train=True, download=True, 
                        transform= transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64,
    shuffle=True,
)

validation_loader = DataLoader(
    datasets.MNIST("E-Learning/NYU-DL/data/MNIST", train=False, download=True, 
                        transform= transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=256,
    shuffle=False,
)

## Modeiling
---
### FC Model:

In [9]:
class FCModel(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(FCModel, self).__init__()
        self.input_size = input_size
        self.n_hidden = n_hidden
        self.output_size = output_size 
        self.network = nn.Sequential(
            nn.Linear(self.input_size, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.output_size),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)

### CNN Model:

In [21]:
class CNNModel(nn.Module):
    def __init__(self, n_channels, output_size):
        super(CNNModel, self).__init__()
        self.n_channels = n_channels
        self.output_size = output_size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_channels//2, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=n_channels//2, out_channels=n_channels, kernel_size=5)
        self.fc1 = nn.Linear(n_channels*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # input shape = m x 28 x 28 x 1
        x = self.conv1(x) # m x 24 x 24 x n_channels//2
        x = F.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x) # m x 12 x 12 x n_channels//2
        x = self.conv2(x) # m x 8 x 8 x n_channels
        x = F.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x) # 4 x 4 x n_channels
        x = nn.Flatten()(x) # m x 4 * 4 * n_channels
        x = self.fc1(x) # m x 50
        x = F.relu(x)
        x = self.fc2(x) # m x 10
        x = F.log_softmax(x)
        return x

Now that we have defined the models, let us define the training and validation functions.

In [22]:
model = CNNModel(n_channels=32, output_size=10)
for batch in train_loader:
    print(model(batch[0]))
    break

tensor([[-2.2344, -2.1492, -2.5311, -2.2290, -2.3330, -2.4834, -2.4807, -2.4725,
         -2.0854, -2.1508],
        [-2.2304, -2.1740, -2.5364, -2.2440, -2.3378, -2.4478, -2.3426, -2.5248,
         -2.0791, -2.2127],
        [-2.2306, -2.2396, -2.5738, -2.2462, -2.2649, -2.5076, -2.3959, -2.4858,
         -2.0248, -2.1865],
        [-2.1066, -2.2707, -2.5409, -2.3072, -2.3547, -2.5582, -2.3257, -2.5003,
         -1.9915, -2.2242],
        [-2.1488, -2.2712, -2.5953, -2.1520, -2.2359, -2.5528, -2.4635, -2.4429,
         -2.1243, -2.1805],
        [-2.1834, -2.3102, -2.4962, -2.2219, -2.2615, -2.4745, -2.3618, -2.4730,
         -2.0975, -2.2298],
        [-2.1170, -2.1885, -2.6078, -2.2726, -2.4754, -2.5444, -2.3768, -2.5362,
         -2.0277, -2.0835],
        [-2.2303, -2.3082, -2.4066, -2.3005, -2.3585, -2.4630, -2.2921, -2.4526,
         -2.0207, -2.2702],
        [-2.2488, -2.2242, -2.6433, -2.1986, -2.3186, -2.4409, -2.3583, -2.5490,
         -2.0338, -2.1614],
        [-2.1405, -

<ipython-input-21-2d2e233a5ffd>:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
